# Análise de Probabilidade de Retorno e Credit Scoring
Este notebook segue o exercício proposto com base na teoria de modelos de classificação binária, utilizando a base de doações da Nota Fiscal Paulista.

## 1. Carregamento da base e filtro a partir de 2020

In [ ]:

import pandas as pd

# Carregar base
df_nfp = pd.read_pickle("base_nfp.pkl")

# Filtrar dados a partir de janeiro de 2020
df_2020 = df_nfp[df_nfp["Data Emissão"] >= "2020-01-01"].copy()
df_2020["flag_retorno"] = (df_2020["Retorno"] > 0).astype(int)
df_2020.head()


## 2. Proporção de Notas com Retorno por Categoria

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Agrupamento por categoria
proporcao_retorno = (
    df_2020.groupby("categoria")["flag_retorno"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

# Gráfico
plt.figure(figsize=(12, 6))
sns.barplot(data=proporcao_retorno, x="flag_retorno", y="categoria", palette="viridis")
plt.xlabel("Proporção de Notas com Retorno > 0")
plt.ylabel("Categoria do Estabelecimento")
plt.title("Proporção de Notas com Retorno por Categoria (a partir de 2020)")
plt.tight_layout()
plt.show()


## 3. Cálculo do Weight of Evidence (WoE)

In [ ]:

import numpy as np

# Função WoE e IV
def calculate_woe_iv(df, feature, target):
    eps = 0.0001
    temp = pd.crosstab(df[feature], df[target])
    temp.columns = ['No_Event', 'Event'] if 0 in temp.columns else ['Event', 'No_Event']
    temp['Dist_Event'] = temp['Event'] / temp['Event'].sum()
    temp['Dist_No_Event'] = temp['No_Event'] / temp['No_Event'].sum()
    temp['WoE'] = np.log((temp['Dist_Event'] + eps) / (temp['Dist_No_Event'] + eps))
    temp['IV'] = (temp['Dist_Event'] - temp['Dist_No_Event']) * temp['WoE']
    return temp.reset_index(), temp['IV'].sum()

# Calcular WoE e IV
woe_df, iv_categoria = calculate_woe_iv(df_2020, 'categoria', 'flag_retorno')
woe_df_sorted = woe_df.sort_values(by='WoE', ascending=False)
woe_df_sorted


## 4. Gráfico do WoE por Categoria

In [ ]:

# Gráfico de WoE
plt.figure(figsize=(12, 6))
sns.barplot(data=woe_df_sorted, x='WoE', y='categoria', palette='coolwarm')
plt.title("WoE por Categoria de Estabelecimento")
plt.xlabel("Weight of Evidence (WoE)")
plt.ylabel("Categoria")
plt.tight_layout()
plt.show()


## 5. Information Value (IV) da variável `categoria`

In [ ]:
print(f'IV da variável categoria: {iv_categoria:.3f}')

## 6. Conclusão


### 📌 Conclusão Técnica

A partir da análise dos dados a partir de 2020, verificou-se que a variável **`categoria`** dos estabelecimentos apresenta variações significativas na proporção de notas com **retorno financeiro positivo** (`Retorno > 0`). Isso indica que essa variável pode ser utilizada como um **atributo discriminativo** em modelos de classificação binária voltados à predição de retorno.

As categorias **Restaurantes** e **Alimentos** destacaram-se com os maiores valores de **Weight of Evidence (WoE)**, evidenciando forte associação com o evento de interesse. Já categorias como **Varejo** e **Vestuário** mostraram valores negativos de WoE, sugerindo menor propensão a gerar créditos.

O **Information Value (IV)** da variável `categoria` foi de **0.278**, indicando um **poder preditivo moderado** de acordo com os critérios clássicos:

- IV < 0.02: Sem valor preditivo  
- 0.02 ≤ IV < 0.1: Fraco  
- 0.1 ≤ IV < 0.3: Moderado  
- IV ≥ 0.3: Forte

Portanto, a variável `categoria` é **estatisticamente relevante** e pode ser utilizada na construção de um **scorecard logístico** ou qualquer modelo de classificação que se beneficie de transformações baseadas em WoE.

---

### 🔍 Próximos Passos Recomendados

- Incluir variáveis contínuas e discretizá-las em faixas para aplicar WoE;
- Realizar validação cruzada com modelos logísticos treinados com e sem WoE;
- Avaliar estabilidade temporal da variável `categoria` (Population Stability Index – PSI);
- Estender o modelo com outras variáveis comportamentais e demográficas, se disponíveis.
